In [1]:
import keras
import pickle as pkl
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import optimizers
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier as KNN
# from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error,make_scorer
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

C:\Users\faizaan09\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open("./data/umm_balanced.pkl","rb") as pk:
    features, labels = pkl.load(pk)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.20, random_state = 42)

scaler = StandardScaler()
scaler.fit(X_train)
sc_train = scaler.transform(X_train)
sc_test = scaler.transform(X_test)

In [34]:
epochs = 100
n_dim = X_train.shape[1]
n_hidden_units_one = 400
n_hidden_units_two = 800
sd = 1 / np.sqrt(n_dim)
learning_rate = 0.002

model = Sequential([
    Dense(n_hidden_units_one, input_shape = (n_dim,), kernel_initializer = keras.initializers.RandomNormal(mean=0, stddev=sd)),
    Activation('sigmoid'),
    Dense(n_hidden_units_two, kernel_initializer = keras.initializers.RandomNormal(mean=0, stddev=sd)),
    Activation('sigmoid'),
    Dense(1),
    Activation('sigmoid')
])
adam = optimizers.Nadam(lr=learning_rate)# Adam(lr=learning_rate)
model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics=['accuracy'])

model.fit(sc_train,y_train,epochs = epochs, batch_size = 32, validation_data=(sc_test,y_test))

score = model.evaluate(sc_test,y_test, batch_size = 32)
print (score)

Train on 3200 samples, validate on 800 samples
Epoch 1/100
3200/3200 [==============================] - 2s 640us/step - loss: 0.6724 - acc: 0.6303 - val_loss: 0.6039 - val_acc: 0.6837
Epoch 2/100
3200/3200 [==============================] - 1s 418us/step - loss: 0.5640 - acc: 0.7109 - val_loss: 0.5582 - val_acc: 0.7300
Epoch 3/100
3200/3200 [==============================] - 2s 587us/step - loss: 0.5326 - acc: 0.7366 - val_loss: 0.5557 - val_acc: 0.7175
Epoch 4/100
3200/3200 [==============================] - 1s 465us/step - loss: 0.5207 - acc: 0.7406 - val_loss: 0.5453 - val_acc: 0.7325
Epoch 5/100
3200/3200 [==============================] - 1s 435us/step - loss: 0.5011 - acc: 0.7522 - val_loss: 0.5397 - val_acc: 0.7400
Epoch 6/100
3200/3200 [==============================] - 1s 427us/step - loss: 0.4769 - acc: 0.7622 - val_loss: 0.5512 - val_acc: 0.7488
Epoch 7/100
3200/3200 [==============================] - 1s 434us/step - loss: 0.4456 - acc: 0.7800 - val_loss: 0.5033 - val_acc: 0

3200/3200 [==============================] - 1s 447us/step - loss: 0.0124 - acc: 0.9959 - val_loss: 1.1322 - val_acc: 0.8450
Epoch 61/100
3200/3200 [==============================] - 1s 436us/step - loss: 0.0114 - acc: 0.9956 - val_loss: 1.0956 - val_acc: 0.8500
Epoch 62/100
3200/3200 [==============================] - 1s 437us/step - loss: 0.0070 - acc: 0.9978 - val_loss: 1.2484 - val_acc: 0.8300
Epoch 63/100
3200/3200 [==============================] - 1s 427us/step - loss: 0.0061 - acc: 0.9972 - val_loss: 1.1620 - val_acc: 0.8438
Epoch 64/100
3200/3200 [==============================] - 1s 441us/step - loss: 0.0085 - acc: 0.9963 - val_loss: 1.3146 - val_acc: 0.8287
Epoch 65/100
3200/3200 [==============================] - 1s 436us/step - loss: 0.0355 - acc: 0.9900 - val_loss: 1.1197 - val_acc: 0.8225
Epoch 66/100
3200/3200 [==============================] - 1s 433us/step - loss: 0.0397 - acc: 0.9853 - val_loss: 1.1121 - val_acc: 0.8337
Epoch 67/100
3200/3200 [=======================

In [3]:
# svc = svm.SVC()

# svc_params = {'kernel':('rbf','linear','poly', 'sigmoid'),'C': [0.1,1,10], 'tol' : [1e-3,1e-6]} 

# svc = GridSearchCV(svm.SVC(), svc_params, scoring = 'accuracy', n_jobs = 8,cv = 5, verbose=1)

svc = svm.SVC(kernel='rbf', C = 10, tol=1e-1, gamma = 0.1,max_iter=1200)

svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)

print(metrics.accuracy_score(svc.predict(X_train), y_train), metrics.accuracy_score(y_test, y_pred))

C:\Users\faizaan09\Anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1200).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


1.0 0.88


(kernel='rbf', C = 10, tol=1e-9, gamma = 0.1)
1.0 0.88125

In [ ]:

rf = RandomForestClassifier(n_jobs=-1)

rf_params = {'n_estimators' : [200,300,400],'max_depth':[100,150,300]}

rf = GridSearchCV(RandomForestClassifier(n_jobs=-1), rf_params, scoring = 'accuracy', n_jobs = 1,cv = 5, verbose=1)

# rf = RandomForestClassifier(n_estimators=100, max_depth=20, n_jobs=-1)
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)

print(metrics.accuracy_score(rf.predict(X_train), y_train), metrics.accuracy_score(y_test, y_pred))


In [11]:
rf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [18]:

knn_params = {'n_neighbors': [10,20,50], 'weights': ('uniform','distance'), 
              'algorithm' :('ball_tree', 'kd_tree', 'brute'),
              'p': [1,2,3]
             }

neigh = GridSearchCV(KNN(n_jobs=-1), knn_params, scoring = 'accuracy', n_jobs = 1,cv = 5, verbose=1)

neigh.fit(X_train, y_train) 

y_pred = neigh.predict(X_test)
print(metrics.accuracy_score(neigh.predict(X_train), y_train), metrics.accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=1)]: Done 270 out of 270 | elapsed: 18.0min finished


1.0 0.8175


In [20]:
neigh.best_estimator_

KNeighborsClassifier(algorithm='ball_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=50, p=1,
           weights='distance')